In [2]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

In [40]:
df = pd.read_csv('../data/1ySOLdata1hAllHassInd.csv')

In [41]:
def process_data(data, window_size, coin):


    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
    data.set_index('timestamp', inplace=True)
    # pd.set_option('future.no_silent_downcasting', True)

    window_size = window_size
    data_trimmed = data.copy()
    data_trimmed.loc[:, 'signal'] = 'SignalNone'

    rolling_max = data_trimmed.loc[:,'price'].rolling(window=2*window_size+1, center=True, min_periods=1).max()
    rolling_min = data_trimmed.loc[:,'price'].rolling(window=2*window_size+1, center=True, min_periods=1).min()

    is_peak = (data_trimmed.loc[:, 'price'] == rolling_max)

    is_low = (data_trimmed.loc[:, 'price'] == rolling_min)

    data_trimmed.loc[is_peak, 'signal'] = 'SignalShort'
    data_trimmed.loc[is_low, 'signal'] = 'SignalLong'
    df = data_trimmed.copy()

    def filter_pivots(data):
      df_filtered = df[df['signal'] != 'SignalNone']


      for i in range(1, len(df_filtered)):
          current_signal = df_filtered.iloc[i]['signal']
          previous_signal = df_filtered.iloc[i - 1]['signal']
          current_close = df_filtered.iloc[i]['price']
          previous_close = df_filtered.iloc[i - 1]['price']

          if current_signal == previous_signal:
              if current_signal == 'SignalLong':
                  if previous_close > current_close:
                      df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
                  else:
                      df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'
              elif current_signal == 'SignalShort':
                  if previous_close < current_close:
                      df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
                  else:
                      df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'
          elif current_signal != previous_signal:
              if current_signal == 'SignalLong':
                  if previous_close < current_close:
                      df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
                      df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'
              elif current_signal == 'SignalShort':
                  if previous_close > current_close:
                      df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
                      df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'

      return df_filtered



    filter_1 = filter_pivots(df)

    df.update(filter_1)
    next_filter = df[['Close', 'signal']].copy()

    filter_2 = filter_pivots(next_filter)
    df.update(filter_2)


    df_fixed = df.copy()
    df_fixed.loc[:,'signal'] = df_fixed.loc[:,'signal'].replace({'SignalLong': 2, 'SignalShort': 0, 'SignalNone': 1})
    df_fixed = df_fixed.ffill()

    feature_names = [col for col in df_fixed.columns if col != 'signal']

    # Save feature names to a JSON file
    # with open(f"/content/models/{coin}_feature_names.json", 'w') as f:
    #     json.dump(feature_names, f)

    return df_fixed

In [42]:
dfcopy = df.copy()
dfpro = process_data(dfcopy, 50, 'SOL')

/var/folders/8h/kn141vz15zx75vwjdp__w9fr0000gn/T/ipykernel_72581/3299500085.py:68: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/8h/kn141vz15zx75vwjdp__w9fr0000gn/T/ipykernel_72581/3299500085.py:69: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [48]:
data = vbt.Data.from_data(dfpro)


In [49]:
predict_list = ['Close', 'atr', 'bbands_lower', 'bop', 'cci', 'cmo', 'crsi', 'donchian_middle', 'ema', 'ht_trendline', 'ht_trendmode', 'ichi', 'kri', 'macdfix_macd', 'sar', 'stochrsi_fastK', 't3', 'tsi', 'willr']
X = data.data['symbol'][predict_list]
# print(X)

y = data.data['symbol']['signal']
y

timestamp
2023-03-12 13:00:00+00:00    1
2023-03-12 14:00:00+00:00    1
2023-03-12 15:00:00+00:00    1
2023-03-12 16:00:00+00:00    2
2023-03-12 17:00:00+00:00    1
                            ..
2024-03-12 22:00:00+00:00    1
2024-03-12 23:00:00+00:00    1
2024-03-13 00:00:00+00:00    1
2024-03-13 01:00:00+00:00    1
2024-03-13 02:00:00+00:00    1
Freq: h, Name: signal, Length: 8798, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
split_percentage = 0.9
split = int(split_percentage*len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-split_percentage)


In [51]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,            
                                criterion='gini', 
                                max_depth=None, 
                                min_samples_split=2, 
                                min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, 
                                max_leaf_nodes=None, 
                                min_impurity_decrease=0.0, 
                                bootstrap=True, 
                                oob_score=True, 
                                n_jobs=-1,
                                random_state=42).fit(X_train, y_train)

y_pred = clf.predict(X_test)


In [52]:
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score
# report = classification_report(y_test, y_pred)
# accuracy = accuracy_score(y_test, y_pred)
# accuracy

In [53]:
df_split = data.data['symbol'][split:].copy()
df_split.loc[:, "signal"] = y_pred
signal = df_split['signal']
entries = signal == 2
exits = signal == 0

In [54]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [55]:
pf.plot({"orders", "cum_returns"}, settings=dict(bm_returns=False)).show()

In [184]:
pf.stats()

Start                         2023-05-26 21:00:00+00:00
End                           2023-12-31 06:00:00+00:00
Period                                218 days 10:00:00
Start Value                                       100.0
Min Value                                     92.974143
Max Value                                    593.788665
End Value                                    491.085838
Total Return [%]                             391.085838
Benchmark Return [%]                         421.028013
Total Time Exposure [%]                       69.534529
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                               31.56876
Max Drawdown Duration                 101 days 21:00:00
Total Orders                                          5
Total Fees Paid                                     0.0
Total Trades                                          3
Win Rate [%]                                       50.0
Best Trade [%]                                46

In [185]:
# import joblib
# def save_model(model, model_filename='trained_model.joblib'):
#     joblib.dump(model, model_filename)
#     print(f'Model saved as {model_filename}')
# save_model(clf)